In [1]:
from geopy.geocoders import Nominatim
from shapely.ops import polygonize
from shapely import geometry
import pandas as pd
import numpy as np
#import geopandas
import overpy
from shapely.ops import linemerge
import requests
import json

In [5]:
class GeoSpaceData():    
    
    def get_place_ID(self, place_name):
        geolocator = Nominatim(user_agent="city_compare")
        geoResults = geolocator.geocode(place_name, exactly_one=False, limit=3, timeout = 600)

        for r in geoResults:
            if r.raw.get("osm_type") == "relation":
                city = r
                break

        place_ID = int(city.raw.get("osm_id")) + 3600000000
        return place_ID
    
    
    def get_places_overpy(self, place_name,key,value):    
        place_ID = self.get_place_ID(place_name)
        api = overpy.Overpass()
        result = api.query("""
            [out:json][timeout:500];
            area(%s)->.searchArea;
            (
              node[%s=%s](area.searchArea);
              way[%s=%s](area.searchArea);
              relation[%s=%s](area.searchArea);
            );
            out geom;
            >;
            out skel qt;
            """ % (place_ID,key,value,key,value,key,value))
        return result
    
        
    def get_places_overpass(self, place_name,key,value):    
        place_ID = self.get_place_ID(place_name)
        print(value)
        api = overpy.Overpass()
        result = api.query("""
            [out:json][timeout:100];
            area(%s)->.searchArea;
            (
              node[%s=%s](area.searchArea);
              way[%s=%s](area.searchArea);
              relation[%s=%s](area.searchArea);
            );
            out geom;
            >;
            out skel qt;
            """ % (place_ID,key,value,key,value,key,value))
        return result
    

    def to_polygon(self, geom, geom_size):
        if geom_size > 1:
            try:
                geom = geometry.Polygon(linemerge(geom))
            except:
                is_first = True
                for LineString in geom:
                    if is_first:
                        try:
                            geom = geometry.Polygon(LineString)
                        except:
                            geom = LineString
                        is_first = False
                    try:
                        contained = geometry.Polygon(LineString)
                    except:
                        contained = LineString
                    if not geom.contains(contained):
                        geom = contained           
        else:
            geom = geometry.Polygon(geom)
        return geom
    
    def get_result(self, place_name,key,value):
        
        return self.get_places_overpass(place_name,key,value)
                
        #return self.get_places_overpy(place_name,key,value)
        
    
    def get_places(self, result,key,value,tags=("name","geom")):
        
       # result = self.get_places_overpy(place_name,key,value)
        
        df = pd.DataFrame(columns=tags)
        
        #print(type(result))
        #print(len(result.nodes))
        #print(len(result.ways))
        #print(len(result.relations))
        
        print(len(result.nodes)+len(result.ways)+len(result.relations))
        
        for n in result.nodes:
            line = []     
            #if n.tags.get(value,np.nan) is not np.nan:   
            for t in tags:
                if t != "geom":
                    line.append(n.tags.get(t, np.nan))
                else:
                    line.append(geometry.Point([n.lon, n.lat]))

            df = df.append(pd.DataFrame([line],columns=tags))

        for w in result.ways:
            line = []
            #if w.tags.get("name",np.nan) is not np.nan:

            for t in tags:
                if t != "geom":
                    line.append(w.tags.get(t, np.nan))
                else:
                    try:
                        line.append(geometry.Polygon([[p.lon, p.lat] for p in w.nodes]))
                    except:
                        try:
                            line.append(geometry.Polygon(linemerge([[p.lon, p.lat] for p in w.nodes])))
                        except:
                            line = None
            if line != None:
                df = df.append(pd.DataFrame([line],columns=tags))


        for r in result.relations:

            line = []
            for t in tags:
                if t != "geom":
                    line.append(r.tags.get(t, np.nan))
            geom = geometry.LineString()
            cont = 0
            #print(r.tags.get('name',np.nan))
            for m in r.members:
                coords = []
                if m.geometry and (m.role == "outer" or m.role == ""):
                    for p in m.geometry:           
                        coords.append([p.lon, p.lat])
                    cont+=1
                    
                geom = geom.union(geometry.LineString(coords))
                
               # dfGeo = geopandas.GeoDataFrame(geometry=[geom])
              #  dfGeo.crs = from_epsg(4326)
               # mapOSM = folium.Map(location=[-19.8752282, -43.9688723],zoom_start=14)
              #  folium.GeoJson(dfGeo).add_to(mapOSM)
               # display(mapOSM)

            line.append(self.to_polygon(geom,cont))
            df = df.append(pd.DataFrame([line],columns=tags))
        
        df["key"] = key
        df["value"] = value
        return df.reset_index(drop=True)    

### Coleta dos Motivos de Parada

In [27]:
label = {"amenity":"bus_station","highway":"traffic_signals","highway":"bus_stop"}

In [6]:
geo = GeoSpaceData()
key = "amenity"
value = "bus_station"
result = geo.get_result("Curitiba",key,value)
df = geo.get_places(result,key,value)
df

bus_station
691


C:\ProgramData\Anaconda3\lib\site-packages\shapely\geometry\polygon.py:539: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  ret = geos_linearring_from_py(shell)
C:\ProgramData\Anaconda3\lib\site-packages\shapely\geometry\polygon.py:539: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  ret = geos_linearring_from_py(shell)
<ipython-input-5-141eb2a48bba>:59: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for LineString in geom:


,name,geom,key,value
0,Terminal Campo Comprido,POINT (-49.3466482 -25.4415818),amenity,bus_station
1,Terminal Hauer,POINT (-49.2470343 -25.4819118),amenity,bus_station
2,Terminal Hauer,POINT (-49.2468834 -25.4817788),amenity,bus_station
3,Guilhermina - 561,POINT (-49.2773758 -25.4874937),amenity,bus_station
4,NaN,POINT (-49.3305895 -25.5757062),amenity,bus_station
...,...,...,...,...
686,Terminal Boa Vista,"POLYGON ((-49.2404556 -25.3934416, -49.24041 -...",amenity,bus_station
687,Terminal Caiuá,"POLYGON ((-49.3503646 -25.4836553, -49.3503808...",amenity,bus_station
688,Terminal Capão Raso,"POLYGON ((-49.2931457 -25.4916271, -49.2931019...",amenity,bus_station
689,Terminal Fazendinha,"POLYGON ((-49.3263873 -25.4773491, -49.3277361...",amenity,bus_station


In [45]:
df.to_csv("osm/"+value+".csv")

### Coleta das funcionalidades 

In [47]:
geo = GeoSpaceData()
def generates_functional_area(file,tags):
    df = pd.DataFrame(columns=["name","geom","key","value"])
    print(file)
    for t in tags: 
        print(t["key"],t["value"])
        result = geo.get_result("Curitiba, Brasil",t["key"],t["value"])
        df = df.append(geo.get_places(result,t["key"],t["value"]))
        #df.to_csv("roi_functional/"+file[5:-5]+"/"+t["key"]+"-"+t["value"]+".csv")
    df.to_csv("roi_functional/"+file[5:-5]+".csv")
    print(len(df))

In [81]:
import glob
for i, file in enumerate(glob.glob('tags/*.json')):
    print("-"+file+"-")
    with open(file) as train_file:
        dict_train = json.load(train_file)
        generates_functional_area(file,dict_train)


-tags\tourism.json-
tags\tourism.json
tourism alpine_hut
alpine_hut
0
tourism apartment
apartment
6
tourism aquarium
aquarium
21
tourism artwork
artwork
70
tourism attraction
attraction
166
tourism camp_pitch
camp_pitch
0
tourism camp_site
camp_site
2
tourism caravan_site
caravan_site
0
tourism chalet
chalet
1
tourism gallery
gallery
49
tourism guest_house
guest_house
9
tourism hostel
hostel
51
tourism hotel
hotel
429


C:\ProgramData\Anaconda3\lib\site-packages\shapely\geometry\polygon.py:539: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  ret = geos_linearring_from_py(shell)
C:\ProgramData\Anaconda3\lib\site-packages\shapely\geometry\polygon.py:539: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  ret = geos_linearring_from_py(shell)
<ipython-input-26-4965f23504d4>:59: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for LineString in geom:


tourism information
information
2
tourism motel
motel
50
tourism museum
museum
215
tourism picnic_site
picnic_site
10
tourism theme_park
theme_park
0
tourism viewpoint
viewpoint
10
tourism wilderness_hut
wilderness_hut
0
tourism zoo
zoo
156
aeroway aerodrome
aerodrome
130
aeroway heliport
heliport
0
aeroway helipad
helipad
103
1478


In [86]:
import glob
rows = []
for i, file in enumerate(glob.glob('tags/*.json')):
    print("-"+file+"-")
    filename = file[5:-5]
    with open(file) as train_file:
        dict_train = json.load(train_file)
        for t in dict_train: 
            rows.append([filename, t["key"],t["value"]])
            
df = pd.DataFrame(rows, columns=('Categoria', 'Key', 'Value'))

-tags\company.json-
-tags\educational.json-
-tags\entertainment.json-
-tags\entertainment2.json-
-tags\food.json-
-tags\government.json-
-tags\shopping.json-
-tags\sport.json-
-tags\sport2.json-
-tags\tourism.json-


In [88]:
df.to_csv('categorias_mapeadas.csv', index=False)

### Coleta dos Bairros

In [36]:
geo = GeoSpaceData()
key = "boundary"
value = "administrative"
result = geo.get_result("Curitiba, Brasil",key,value)
df = geo.get_places(result,key,value)
df

administrative
34753


,name,geom,key,value
0,NaN,POINT (-54.5264199 -25.6286655),boundary,administrative
1,NaN,POINT (-54.527599 -25.6280658),boundary,administrative
2,NaN,POINT (-54.5351092 -25.6219327),boundary,administrative
3,NaN,POINT (-54.5337952 -25.6229406),boundary,administrative
4,NaN,POINT (-54.5316523 -25.6245843),boundary,administrative
...,...,...,...,...
34595,Atuba,"POLYGON ((-49.2194267 -25.3801377, -49.2191466...",boundary,administrative
34596,Xaxim,"POLYGON ((-49.2848309 -25.5073907, -49.2848527...",boundary,administrative
34597,Capão Raso,"POLYGON ((-49.3137888 -25.5165366, -49.313359 ...",boundary,administrative
34598,Pinheirinho,"POLYGON ((-49.3208546 -25.5189195, -49.3205472...",boundary,administrative


In [42]:
df[df.name.notna()].drop_duplicates(subset=["name"]).to_csv("bairros.csv")

In [95]:
b = ["ÁGUA VERDE", "JARDIM SOCIAL","XAXIM","JARDIM DAS AMÉRICAS","GUAÍRA","BAIRRO ALTO","SÃO FRANCISCO",
 "ALTO DA GLÓRIA","ATUBA","CABRAL","LINDÓIA","CAMPO COMPRIDO","CENTRO CÍVICO","JUVEVÊ","HUGO LANGE",
 "CAPÃO DA IMBUIA","SEMINÁRIO","GUABIROTUBA","PRADO VELHO","PAROLIN","PORTÃO","VILA IZABEL","CAMPINA DO SIQUEIRA",
 "BOA VISTA","AHÚ","CRISTO REI","REBOUÇAS","ORLEANS","PILARZINHO","MOSSUNGUÊ","ALTO DA RUA XV","BAIRRO JARDIM BOTÂNICO",
 "BUTIATUVINHA","BARREIRINHA","TINGUI","BOQUEIRÃO","RIVIERA","SÍTIO CERCADO","TARUMÃ","SÃO LOURENÇO","BACACHERI",
 "BIGORRILHO","BOM RETIRO","CAPÃO RASO","HAUER","FANNY","NOVO MUNDO","FAZENDINHA","SANTA QUITÉRIA","CENTRO",
 "ABRANCHES","BATEL","SÃO MIGUEL","AUGUSTA","CAXIMBA","ALTO BOQUEIRÃO","PINHEIRINHO","TATUQUARA","UMBARÁ",
 "CAMPO DE SANTANA","GANCHINHO","CAJURU","UBERABA","TABOÃO","CACHOEIRA","SANTA CÂNDIDA","CIDADE INDUSTRIAL DE CURITIBA",
 "LAMENHA PEQUENA","MERCÊS","SANTO INÁCIO","VISTA ALEGRE","SÃO BRAZ","SÃO JOÃO","CASCATINHA","SANTA FELICIDADE"]

In [96]:
df["name"] = df[df.name.notna()]["name"].apply(lambda x: x if (x.upper() in b) else None)

In [101]:
df[df.name.notna()].reset_index(drop=True).to_csv("bairros.csv",index=False)